In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.special
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.subplot.hspace'] = 0.25
plt.rcParams['figure.subplot.left'] = 0.17
plt.rcParams['axes.labelsize'] = 16


### Diffusion equation

The solution is
$$\frac{1}{2}\left(1+\mathrm{erf}(-\frac{y}{\sqrt{4Dt}})\right)$$
The "standard" definition matches the one of SciPy for [erf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.erf.html).

The derivative is
$$\lambda = - \frac{1}{\sqrt{4\pi Dt}}e^{-y^2/(4Dt)}$$

Both results are shifted by $L_y/2$ in the simulation as the separation
of input channels takes place there.

The time is directly related to the position in the channel via
$$t = \frac{x}{v}$$

In [ ]:
class diffusion(object):
    def __init__(self, D, rho, v_max, Ly):
        self._D = D
        self._rho = rho
        self._v_max = v_max
        self._Ly = Ly
    def c_A(self, x, y):
        """Concentration of A species at position x, y"""
        t_eff = 4*self._D*x/self._v_max
        return self._rho*(1+scipy.special.erf(-(y-self._Ly/2)/np.sqrt(t_eff)))/2
    def lam(self, x, y):
        """y derivative of the concentration at position x, y"""
        t_eff = 4*self._D*x/self._v_max
        return -self._rho/np.sqrt(np.pi*t_eff)*np.exp(-(y-self._Ly/2)**2/t_eff)
    

In [ ]:
# Parameters for "cceq_2.p"
d = diffusion(0.194442848776, 10, 0.1583, 32)

In [ ]:
X, Y = np.meshgrid(np.linspace(0.1, 20, 180),np.linspace(0, 32, 150))

In [ ]:
plt.pcolormesh(X, Y, d.c_A(X, Y))
#plt.xlim(0, 30)
#plt.ylim(0, 32)
plt.colorbar()
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.title('Concentration of A in the cell')

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(121)
plt.pcolormesh(X, Y, d.lam(X, Y))
plt.colorbar()
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.title('y-derivative of $c_A$ - formula')

cprime = np.diff(d.c_A(X, Y), axis=0)/(Y[1,0]-Y[0,0])
plt.subplot(122)
plt.pcolormesh(X[:-1,:], Y[:-1,:], cprime)
plt.colorbar()
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.title('y-derivative of $c_A$ - numerical');


In [ ]:
Y[1,0]-Y[0,0], 32/99

In [ ]:
class d_solution(object):
    def __init__(self, diffusor, x0):
        self._diffusor = diffusor
        self._x0 = x0
        self.k0 = 0
        self.kD = 0
        self.R = 0
    def cB(self, x, y):
        x0, y0 = self._x0
        c0 = self._diffusor.c_A(x0, y0)
        lam = self._diffusor.lam(x0, y0)
        c1 = - self.k0/(self.k0+self.kD)*c0
        c2 = - self.k0/(self.k0+2*self.kD)*lam*self.R
        r = np.sqrt((x-self._x0[0])**2+(y-self._x0[1])**2)
        h = 1/R
        r_inverse = self.R/(np.exp(-h*r)/h+r)
        costheta = (y-y0)/r
        return - c1*R/r - c2*R**2/r**2*costheta

    def cart_c(self, x, y):
        r = np.sqrt(x**2+y**2)
        theta = np.arccos(y/r)
        return self.polar_c(r, theta)
    

In [ ]:
s = d_solution(d, x0=(3, 16))

In [ ]:
k0 = 1
s.k0 = k0
R = 2
R_cut = R*2**(1/6)
s.R = R
s.kD = 4*np.pi*R*s._diffusor._D
beta = 3
eps = 1
def V(r):
    return 4*eps*((R/r)**12-(R/r)**6) + eps


In [ ]:
c = d.c_A(X, Y) - s.cB(X, Y)
r = np.sqrt((X-s._x0[0])**2+(Y-s._x0[1])**2)
mask = r<R_cut
c[mask] = c[mask]*np.exp(-beta*V(r[mask]))
plt.pcolormesh(X, Y, c)
plt.axis([X.min(), X.max(), Y.min(), Y.max()])
plt.colorbar()

In [ ]:
r = np.linspace(0.5, 5, 100)
plt.plot(r, 1/r)
h = 1
plt.plot(r, 1/(np.exp(-h*r)/h+r))
plt.plot(r, V(r))
plt.ylim(0, 10)

In [ ]:
x = np.linspace(-20, 20, 400)
D = 1
t = 5
f = 1/np.sqrt(np.pi*4*D*t)*np.exp(-x**2/(4*D*t))
plt.plot(x, f)
print(np.sum(f)*(x[1]-x[0]))

In [ ]:
scipy.integrate.quad?

In [ ]:
scipy.integrate.quad(lambda x: np.exp(-x**2)*2/np.sqrt(np.pi), -np.inf, 0)